# Apply ImageDataGenerator functionality to Train and Test set

In [1]:
pwd

'/home/wsuser/work'

# Preprocessing

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1)

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='x8uP4_9FnAjnShWI-X95nik-6mfeXKvcMecJ-KpyNvn4',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'fertilizerrecommendationsystemfor-donotdelete-pr-5xnf3hacie9ynt'
object_key = 'Fertilizers_Recommendation_ System_For_Disease_ Prediction.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
pwd

'/home/wsuser/work'

In [ ]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test')

In [7]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/train',target_size=(128,128),batch_size=32,class_mode='categorical')


Found 5384 images belonging to 6 classes.


In [8]:
x_test=test_datagen.flow_from_directory('/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test',target_size=(128,128),
                                          batch_size=32,class_mode='categorical')

Found 1686 images belonging to 6 classes.


# Import the models

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPool2D,Flatten

# Initializing the models

In [10]:
model=Sequential()

# Add CNN Layers

In [11]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [12]:
x_train.class_indices

{'Apple___Black_rot': 0,
 'Apple___healthy': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3,
 'Peach___Bacterial_spot': 4,
 'Peach___healthy': 5}

# Add Pooling layer

In [13]:
model.add(MaxPool2D(pool_size=(2,2)))

# Add Flatten layer

In [14]:
model.add(Flatten())

# Add Dense Layer

In [15]:
model.add(Dense(40, kernel_initializer='uniform',activation='relu'))
model.add(Dense(20, kernel_initializer='random_uniform',activation='relu'))

# Add Output Layer

In [16]:
model.add(Dense(6,activation='softmax', kernel_initializer='random_uniform'))

# Compile the model

In [17]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
len(x_train)

169

In [19]:
5384/32

168.25

# Fit the Model

In [20]:
model.fit_generator(x_train,steps_per_epoch=168,validation_data=x_test,validation_steps=52,epochs=3)

/tmp/wsuser/ipykernel_164/2881909211.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=168,validation_data=x_test,validation_steps=52,epochs=3)


Epoch 1/3
168/168 [==============================] - 94s 557ms/step - loss: 1.1376 - accuracy: 0.5805 - val_loss: 90.1628 - val_accuracy: 0.6719
Epoch 2/3
168/168 [==============================] - 93s 551ms/step - loss: 0.5609 - accuracy: 0.7986 - val_loss: 51.1172 - val_accuracy: 0.8413
Epoch 3/3
168/168 [==============================] - 94s 556ms/step - loss: 0.3945 - accuracy: 0.8612 - val_loss: 170.6686 - val_accuracy: 0.6965


# Save the Model

In [21]:
model.save("fruit.h5")

In [22]:
!tar -zcvf Fertilizer-Recommendation-System-For-Disease-Prediction_new.tgz fruit.h5

fruit.h5


In [23]:
ls -1


'Dataset Plant Disease'/
Fertilizer-Recommendation-System-For-Disease-Prediction_new.tgz
fruit.h5


In [24]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 8.9 MB/s eta 0:00:01


In [25]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"yGaJNrE8L3BQwBbZw2gxotSSCdvSazEIl5eyRuytrnFf"
                  }
client = APIClient(wml_credentials)

In [26]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [28]:
space_uid = guid_from_space_name(client, 'fertilizerrecommendationfordiseaseprediction')
print("Space UID = " + space_uid)

Space UID = d83b1051-df7c-46bb-bb74-1f325d5d6119


In [29]:
client.set.default_space(space_uid)

'SUCCESS'

In [30]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [31]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [32]:
ls

'Dataset Plant Disease'/                                           fruit.h5
 Fertilizer-Recommendation-System-For-Disease-Prediction_new.tgz


In [33]:
model_details = client.repository.store_model(model='Fertilizer-Recommendation-System-For-Disease-Prediction_new.tgz',
    meta_props={
        client.repository.ModelMetaNames.NAME:"CNN",
        client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
    )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [ ]:
model_id

In [ ]:
client.repository.download(model_id,'fruit.tar.gb')

In [ ]:
pwd

# Test the Model

In [26]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np


In [27]:
model = load_model("fruit.h5")

# Test Apple_Healthy Class images

In [59]:
import keras.utils as image

In [60]:
img = image.load_img('Dataset Plant Disease/fruit-dataset/fruit-dataset/test/Apple___healthy/00fca0da-2db3-481b-b98a-9b67bb7b105c___RS_HL 7708.JPG',target_size=(128,128))

In [61]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

In [75]:
pred = np.argmax(model.predict(x),axis=1)

1/1 [==============================] - 0s 24ms/step


In [76]:
pred

array([1], dtype=int64)

In [79]:
index =['Apple___Black_rot','Apple___healthy','Corn_(maize)___Northern_Leaf_Blight','Corn_(maize)___healthy','Peach___Bacterial_spot','Peach___healthy']

In [80]:
print('the given image belogs to=',index[pred[0]])

the given image belogs to= Apple___healthy


# Test Apple Black Rot class images

In [82]:
img = image.load_img('Dataset Plant Disease/fruit-dataset/fruit-dataset/test/Apple___Black_rot/0f3d45f4-e121-42cd-a5b6-be2f866a0574___JR_FrgE.S 2870.JPG',target_size=(128,128))

In [83]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)
print('the given image belogs to=',index[pred[0]])

1/1 [==============================] - 0s 24ms/step
the given image belogs to= Apple___Black_rot


# Test Corn Northern leaf Blight class images

In [84]:
img = image.load_img('Dataset Plant Disease/fruit-dataset/fruit-dataset/test/Corn_(maize)___Northern_Leaf_Blight/00a14441-7a62-4034-bc40-b196aeab2785___RS_NLB 3932.JPG',target_size=(128,128))

In [85]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)
print('the given image belogs to=',index[pred[0]])

1/1 [==============================] - 0s 24ms/step
the given image belogs to= Corn_(maize)___Northern_Leaf_Blight


# Test Corn Healthy class images

In [86]:
img = image.load_img('Dataset Plant Disease/fruit-dataset/fruit-dataset/test/Corn_(maize)___healthy/0a68ef5a-027c-41ae-b227-159dae77d3dd___R.S_HL 7969 copy.jpg',target_size=(128,128))

In [87]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)
print('the given image belogs to=',index[pred[0]])

1/1 [==============================] - 0s 24ms/step
the given image belogs to= Corn_(maize)___healthy


# Test Peach Bacterial spot class images

In [88]:
img = image.load_img('Dataset Plant Disease/fruit-dataset/fruit-dataset/test/Peach___Bacterial_spot/00ddc106-692e-4c67-b2e8-569c924caf49___Rutg._Bact.S 1228.JPG',target_size=(128,128))

In [89]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)
print('the given image belogs to=',index[pred[0]])

1/1 [==============================] - 0s 24ms/step
the given image belogs to= Peach___Bacterial_spot


# Test Peach Healthy class images

In [90]:
img = image.load_img('Dataset Plant Disease/fruit-dataset/fruit-dataset/test/Peach___healthy/1a07ce54-f4fd-41cf-b088-144f6bf71859___Rutg._HL 3543.JPG',target_size=(128,128))

In [91]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)
print('the given image belogs to=',index[pred[0]])

1/1 [==============================] - 0s 31ms/step
the given image belogs to= Apple___healthy
